# Pipeline to run all files
* ~~All scripts currently output Excel files because we were working independently~~
* ~~<b>We'll be changing this so that each script passes dataframes until the end of the pipeline.</b>~~
* Our plan is to not store anything (refresh daily) but need to consider how to deal with taking in user feedback
* Good copy

In [23]:
#Gather Articles
import NewsAPI as news
import os

#delete any old sorted article files
#if os.path.exists("articles.csv"):
#    os.remove("articles.csv")

#inputs in order: set manual date (yes=1/no=0, no = last 24 hours of news), manual start date as "YYYY-MM-DD"(leave empty if 0), manual end date (leave empty if 0, companies to pull (6= all 19 companies)
articleDB = news.main(1,"2019-02-25","2019-03-01",6) #output is called 'NewsAPIOutput.xlsx' in Python Scripts > Data folder

Gathering articles on (Gap Inc) OR (Foot Locker) OR (L Brands) OR Macerich OR Kimco OR TJX OR CVS OR (Home Depot) OR (Best Buy) OR (Lowe's) OR Walmart OR (Target's) OR TGT OR Amazon OR Kroger OR Walgreens OR Kohl's OR (Dollar General) OR (Bed Bath and Beyond) OR Safeway from: 2019-02-25 to 2019-03-01
997


In [24]:
#Article Cleaning (must pip install tqdm first (only once) to run)
import dataClean as dc
articleDB = dc.DataClean(articleDB)
#articleDB = DataCleann(articleDB)
articleDB.columns

Index(['title', 'description', 'url', 'source', 'date', 'content',
       'origContent', 'contentWithStops'],
      dtype='object')

In [25]:
#Feature Selection and Binary Article Encoding
import FeatureEncoding as fe
contentBinaryMatrix = fe.encoding(0, df=articleDB, text_col='content', norm='wnLemm')
titleBinaryMatrix = fe.encoding(0, df=articleDB, text_col = 'title', norm='wnLemm')

content
wnLemm
Binary Encoding
True
title
wnLemm
Binary Encoding
True


In [26]:
#Logistic Regression Classifier + Article Ranking, complete final file is called 'results_encoding.xlsx'
import logReg as lr
articleDB = lr.runLogReg(titleBinaryMatrix, contentBinaryMatrix, articleDB)
#articleDB = runLogReg(titleBinaryMatrix, contentBinaryMatrix, articleDB)

Index([0, 1, 'article_id', 'prediction'], dtype='object')
                                               title  \
0    Macy’s and Home Depot flag slowing sales growth   
1   Short sellers get set for more pain in US retail   
2         And America's least favorite stores are...   
3  L Brands plans to close roughly 53 Victoria's ...   
4  Chinese firm behind the 'Amazon Coat' hits jac...   

                                         description  
0  US retailers report mixed earnings as investor...  
1  Industry divides into winners and losers from ...  
2  When it comes to satisfying their customers, m...  
3  L Brands, owner of Victoria's Secret and Bath ...  
4  When Kevin Chiu left his job in 2012 to try hi...  


In [27]:
# This code extracts and ranks "tags" + counts frequency of tag mentions in articles 
"""
Inputs: 
    1) articleDB - uses column 'content'
    2) (optional) - tag type (could be 'ngrams'{unlimited}, 'bigrams'{terms with up to 2 words}, or 'unigrams'{single terms})
        - default is 'bigrams'
        - future work: add in noun phrases, named entities 
Outputs:
    1) articleDB = articleDB with appended columns "tags" and "tags_top_5"
    2) trendingTermsDB = keyterms by # article mentions
"""
import ContextExtraction as ce
articleDB, trendingTermsDB = ce.retrieveContext(articleDB)

100%|████████████████████████████████████████████████████████████████████████████████| 578/578 [04:15<00:00,  1.25it/s]


In [28]:
import EncodingforRecommender as rec
articleDB = rec.recommender(articleDB)

tifidf Encoding
bin Encoding


100%|███████████████████████████████████████████████████████████████████████████████| 578/578 [00:04<00:00, 142.26it/s]


tf Encoding


100%|███████████████████████████████████████████████████████████████████████████████| 578/578 [00:03<00:00, 152.08it/s]


          two   biggest     names    retail      macy      home     depot  \
0    0.006757  0.009901  0.035714  0.041667  0.333333  0.034247  0.153846   
1    0.000000  0.039604  0.000000  0.097222  0.083333  0.006849  0.038462   
2    0.003378  0.000000  0.035714  0.041667  0.000000  0.000000  0.000000   
3    0.000000  0.000000  0.000000  0.013889  0.000000  0.000000  0.000000   
4    0.000000  0.000000  0.000000  0.013889  0.000000  0.000000  0.000000   
5    0.000000  0.000000  0.000000  0.013889  0.000000  0.006849  0.000000   
6    0.000000  0.000000  0.000000  0.027778  0.000000  0.006849  0.000000   
7    0.000000  0.009901  0.000000  0.000000  0.000000  0.000000  0.000000   
8    0.003378  0.000000  0.000000  0.013889  0.000000  0.000000  0.000000   
9    0.000000  0.000000  0.000000  0.041667  0.083333  0.006849  0.000000   
10   0.000000  0.000000  0.000000  0.027778  0.000000  0.006849  0.000000   
11   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

[578 rows x 22197 columns]


In [29]:
import frontPage as fp
frontpage = fp.FrontPage(articleDB, trendingTermsDB)

In [30]:
import pandas as pd
pd.DataFrame.to_excel(articleDB, 'Main Version-good-25-01.xlsx')

In [31]:
import winsound
winsound.Beep(600, 600)
winsound.Beep(800, 600)
winsound.Beep(600, 600)
winsound.Beep(800, 600)
winsound.Beep(600, 600)

In [32]:
import os
from os import listdir
from os.path import isfile, join
from pathlib import Path
import pandas as pd
import re, string

def DataCleann(articleDf):
    #remove blanks (NaN)
    articleDf = articleDf.dropna(subset = ['content', 'title']) 

    #remove blocked articles without content
    articleDf = articleDf[articleDf.content.str.contains('Your usage has been flagged', case=False) == False]
    articleDf = articleDf[articleDf.content.str.contains('To continue, please click the box', case=False) == False]

    # vidoes/ads/commentary
    articleDf = articleDf[articleDf.description.str.contains('The "Fast Money" traders share their first moves for the market open.', case=False) == False]
    articleDf = articleDf[articleDf.description.str.contains('stuff we think you', case=False) == False]

    #remove transcripts
    articleDf = articleDf[articleDf.title.str.contains('transcript', case=False) == False]

    #remove cramer
    articleDf = articleDf[articleDf.title.str.contains('cramer', case=False) == False]

    #remove articles with less than words which is the lower end of the boxplot
    articleDf = articleDf[articleDf['content'].str.split().str.len() > 300]

    #remove duplicates
    # by self-identified repeat
    articleDf = articleDf[articleDf.title.str.contains('rpt', case=False) == False]
    # by title
    articleDf = articleDf.drop_duplicates(subset=['title'], keep='first')
    # by content
    articleDf = articleDf.drop_duplicates(subset=['content'], keep='first')
    # by decription
    articleDf = articleDf.drop_duplicates(subset=['description'], keep='first')

    articleDf = articleDf.reset_index(drop=True)

    # CLEAN ORIGINAL CONTENT
    articleDf['origContent'] = articleDf['content'] 

    #Remove nonsense sentence from original content pull
    for i in articleDf.index:
        article = articleDf['origContent'].iloc[i].split('\r\n')

        # remove lines with no period
        article[:] = [sentence for sentence in article if '.' in sentence]
        # remove lines with less than 5 words
        article[:] = [sentence for sentence in article if len(sentence.split())>5]
        # remove photo credits
        article[:] = [sentence for sentence in article if not('Photo' in sentence)]
        blackList = ['get breaking news','click here','write to','subscribe','read more','read or share'
                     ,'reporting by','twitter, instagram','comment','copyright','©', 'fox', 'you', 'sign up', 'your inbox']
        # remove lines with terms that are associated with useless sentences
        article[:] = [sentence for sentence in article if not any(term in sentence.lower() for term in blackList)]
        try:
            article[0] = '<p>'+article[0]
            article[len(article)-1] = article[len(article)-1]+'</p>'
        except:
            continue
        
        articleDf.at[i,'origContent']='</p> <p>'.join(article)

    #Remove videos from cnbc links
    pat_cnbcVid = re.compile('div &gt; div\.group &gt; p:first-child"&gt;')
    articleDf['origContent'] = list(map(lambda x: pat_cnbcVid.sub('', x), articleDf['origContent']))
    pat_vid = re.compile('gt;')
    articleDf['origContent'] = list(map(lambda x: pat_vid.sub('', x), articleDf['origContent']))
    #Remove amp;
    pat_amp = re.compile('amp;')
    articleDf['origContent'] = list(map(lambda x: pat_amp.sub('', x), articleDf['origContent']))    
    
    # CLEAN CONTENT FOR FEATURE SELECTION articleDf['content'] AND CONTEXT EXTRACTION articleDf['contentWithStops'] 

    #Remove time
    pat_time = re.compile('[0-9]{0,2}:?[0-9]{1,2}\s?[aApP]\.?[mM]\.?')
    articleDf['content'] = list(map(lambda x: pat_time.sub(' ', x), articleDf['origContent']))

    #Remove urls
    pat_url = re.compile('[a-z]+?[.]?[a-z]+?[.]?[a-z]+[.]?[\/\/]\S+')
    articleDf['content'] = list(map(lambda x: pat_url.sub('URL', x), articleDf['content']))
    pat_https = re.compile('https://')
    articleDf['content'] = list(map(lambda x: pat_https.sub('', x), articleDf['content']))
    pat_http = re.compile('http://')
    articleDf['content'] = list(map(lambda x: pat_http.sub('', x), articleDf['content']))
    
    #Remove characters that don't separate a sentence or aren't $ signs
    # FOR context extraction
    # Remove non-ascii chars -- these get cleaned up in 'content' when we remove punctuation
    pat_nonascii = re.compile('[^\x00-\x7f]')
    articleDf['contentWithStops'] = list(map(lambda x: pat_nonascii.sub(' ', x), articleDf['content']))
    #pat_nonStops = re.compile('[^\.\?!,;\$0-9a-zA-Z]+')
    #articleDf['contentWithStops'] = list(map(lambda x: pat_nonStops.sub(' ', x), articleDf['content']))
    
    #Remove stopwords & apply lowercasing
    stopwords = [
        # dates/times
        "january", "february", "march", "april", "may", "june", "july", "august", "september", "october", "november", "december", "jan", "feb","mar", "apr", "jun", "jul", "aug", "oct", "nov", "dec", "monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday", "morning", "evening","today","pm","am","daily", 
        # specific article terms that are useless
        "read", "share", "file", "'s","'t", "photo", "inc", "corp", "group", "inc", "corp", "source", "bloomberg", "cnbc","cnbcs", "cnn", "reuters","bbc", "published", "broadcast","msnbc","ap","said","nbcuniversal","newsletterupgrade","nbc", "news",
        # other useless terms
        "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "co", "inc", "com", "theyve", "theyre", "theres", "heres", "didnt", "wouldn", "couldn", "didn","according", "just", "us", "ll", "times","yes","such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "just", "don", "now", "will", "wasn", "etc", "but", "hello", "welcome", "re","my","wasnt","also","us","the", "a", "of", "have", "has", "had", "having", "hello", "welcome", "yeah", "wasn", "today", "etc", "ext","definitely", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "while", "of", "at", "by", "for", "about", "into", "through", "during", "before", "after", "to", "from", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "just", "don", "now", "will"
    ]
    pat_stopwords = re.compile(r'\b(' + r'|'.join(stopwords) + r')\b\s*')
    articleDf['content'] = list(map(lambda x: pat_stopwords.sub(' ', x), articleDf['content'].str.lower()))

    #Remove single character words
    pat_charLim = re.compile('\s[a-zA-Z]\s')
    articleDf['content'] = list(map(lambda x: pat_charLim.sub(' ', x), articleDf['content']))

    #Remove punctuation 
    # FOR feature selection/encoding
    pat_punctuation = re.compile('[^a-zA-Z]+')
    articleDf['content'] = list(map(lambda x: pat_punctuation.sub(' ', x), articleDf['content']))

    #Remove single characters
    articleDf['content'] = list(map(lambda x: pat_charLim.sub(' ', x), articleDf['content']))

    return articleDf

In [33]:
articleDB

,nonRel,Rel,url,prediction,title,description,source,date,content,origContent,contentWithStops,context,tags,tags_top_5,related_articles
0,0.354992,0.645008,https://www.ft.com/content/a0e21066-39dc-11e9-...,1,Macy’s and Home Depot flag slowing sales growth,US retailers report mixed earnings as investor...,financial-times,2019-02-26T17:56:42Z,two biggest names retail macy home depot cauti...,"<p>Two of the biggest names in US retail, Macy...","Two of the biggest names in US retail, Macy s ...","biggest names, Macy, Home Depot, sales growth,...","biggest names, macy, home depot, sales growth,...","own stores,net sales,profit warning,american c...","90, 62, 1"
1,0.385763,0.614237,https://www.ft.com/content/ba371cd4-366f-11e9-...,1,Short sellers get set for more pain in US retail,Industry divides into winners and losers from ...,financial-times,2019-02-27T04:00:25Z,short sellers made big bets against retailers ...,<p>Short sellers who made big bets against US ...,Short sellers who made big bets against US ret...,"Short sellers, big bets, Amazon, make life, ma...","short sellers, big bets, amazon, make life, ma...","profit warning,lamensdorf,share prices,yoga le...","191, 109, 0"
2,0.405513,0.594487,https://www.nbcnews.com/business/consumer/amer...,1,And America's least favorite stores are...,"When it comes to satisfying their customers, m...",nbc-news,2019-02-26T20:31:18Z,gmt comes satisfying customers s retailers br...,"<p>Feb. 26, 2019, 8:31 PM GMT</p> <p>When it c...","Feb. 26, 2019, GMT When it comes to satisfyi...","Feb., 262019GMT, most U.S. retailers, second y...","most u.s. retailers, second year, annual consu...","second year,customer experience,store staff,ac...","141, 443, 85"
3,0.423063,0.576937,https://www.cnbc.com/2019/02/28/l-brands-plans...,1,L Brands plans to close roughly 53 Victoria's ...,"L Brands, owner of Victoria's Secret and Bath ...",cnbc,2019-02-28T12:45:00Z,l brands plans close roughly victoria secret s...,<p>L Brands plans to close roughly 53 Victoria...,L Brands plans to close roughly 53 Victoria's ...,"Brands, 53Victoria, lingerie retailer, newer r...","lingerie retailer, newer rivals, bath, body wo...","works,body,net sales,share result,newer rivals","188, 74, 63"
4,0.433012,0.566988,https://in.reuters.com/article/china-coat-orol...,1,Chinese firm behind the 'Amazon Coat' hits jac...,When Kevin Chiu left his job in 2012 to try hi...,reuters,2019-02-25T04:28:10Z,jiaxing curl york kevin chiu left job try luck...,"<p>JIAXING, China/NEW YORK (Reuters) - When Ke...","JIAXING, CURL YORK (Reuters) - When Kevin Chiu...","Kevin Chiu, online apparel business, rural Chi...","kevin chiu, online apparel business, rural chi...","local commerce sites,u.s. consumers,fierce com...","556, 156, 506"
5,0.433535,0.566465,https://www.cnbc.com/2019/02/25/wayfair-is-out...,1,"Wayfair is up nearly 70% this year, and charts...",It's a case of David vs. Goliath with Amazon t...,cnbc,2019-02-25T13:25:00Z,epic tale david versus goliath commerce world...,<p>It's an epic tale of David versus Goliath i...,It's an epic tale of David versus Goliath in t...,"epic tale, David, Goliath, commerce world, Way...","epic tale, david, goliath, commerce world, way...","mark tepper,next recession,amazon shares,retai...","480, 191, 109"
6,0.440109,0.559891,https://www.bloomberg.com/news/articles/2019-0...,1,Private Equity Faces Down Brexit Risk as KKR L...,It would take a brave private equity investor ...,bloomberg,2019-02-25T18:27:12Z,customer pushes shopping cart walks towards a...,<p>A customer pushes a shopping cart as she wa...,A customer pushes a shopping cart as she walks...,"pushes shopping cart, Asda supermarket, Wal Ma...","pushes shopping cart, asda supermarket, wal ma...","sainsbury plc,walmart inc.,u.k. retailer,fire ...","15, 235, 56"
7,0.447545,0.552455,https://apnews.com/644841507f3b4bae81f70cc3169...,1,US stocks rise on hopes for a trade deal with ...,"NEW YORK (AP) — U.S. stocks are rising Friday,...",associated-press,2019-03-01T